In [1]:
# Load libraries and setup constansts
from gurobipy import *
import pandas as pd

transport_cost = 1 #eur per km
demand = 100 #per week
dists = pd.read_csv('cities_dist.csv')

In [11]:
def get_dist_model(dists, transport_cost, fixed_cost, demand):
    ### Optimizes distribution network
    # vars:
    # - dists: pandas data frame of distances between cities
    # - transport_cost: transport costs per km
    # - fixed_cost: fixed costs in eur
    # - demand: demand in units
    # returns: a list of distributions centeres to be openeded.
    ###
    
    BIG_CONST = 100000
    cities = list(dists)[1:]
    
    model = Model('Distribution Network')
    model.Params.OutputFlag = 0 #turn-off printing detailed results
    X = model.addVars(cities, vtype=GRB.BINARY, name="DC_opened") 
    A = model.addVars(cities, cities, vtype=GRB.CONTINUOUS, name="Amount_transported") 

    model.addConstrs((A[DC, DA] >= 0 for DC in cities for DA in cities), name = "Transfered amount must be non-negative")
    model.addConstrs(((quicksum(A[DC, DA] for DC in cities) >= demand) for DA in cities), name="Demand must be satisfied")
    model.addConstrs(((quicksum(A[DC, DA] for DA in cities) <= BIG_CONST * X[DC]) for DC in cities), name="Demand is zero if a DC is not oppeneded")    
    obj = quicksum(X[DC] * fixed_cost for DC in cities) + quicksum(quicksum(transport_cost * dists[DC][cities.index(DA)] * A[DC, DA] for DC in cities) for DA in cities)
    model.setObjective(obj, GRB.MINIMIZE)
    model.optimize()
    if model.Status == GRB.OPTIMAL: # if everything worked well
    	opened_cities = [city for city, value in X.iteritems() if value.X > 0]
    	#for v in model.getVars():
        #    if v.X != 0:
        #    	pass    
        #    	print(v.x)
        #    print("%s %f" % (v.Varname, v.X))
    else:
    	opened_cities = ['Not optimal solution found. '] 
    return opened_cities
   
prev = []
for fixed_cost_tmp in range(100000,800000,10000): #range of fixed cost computed    
    DCs = get_dist_model(dists, transport_cost, fixed_cost_tmp, demand)
    if DCs != prev:
        print("For fixed cost of %d eur, following DCs will be opended: %s" % (fixed_cost_tmp, ', '.join(DCs)))
    prev = DCs

For fixed cost of 100000 eur, following DCs will be opended: Athens, Brussels, Hamburg, Madrid, Munich, Naples, Turin
For fixed cost of 110000 eur, following DCs will be opended: Athens, Brussels, Hamburg, Madrid, Munich, Rome, Turin
For fixed cost of 130000 eur, following DCs will be opended: Antwerp, Athens, Genoa, Madrid, Prague, Strasbourg
For fixed cost of 150000 eur, following DCs will be opended: Antwerp, Athens, Madrid, Munich, Turin
For fixed cost of 230000 eur, following DCs will be opended: Antwerp, Madrid, Munich, Turin
For fixed cost of 260000 eur, following DCs will be opended: Antwerp, Madrid, Milan
For fixed cost of 350000 eur, following DCs will be opended: Antwerp, Turin
For fixed cost of 740000 eur, following DCs will be opended: Strasbourg


In [5]:
# optimization model where fixed cost is a parameter of a slider
from ipywidgets import widgets, interact

def get_dist_model_interact(fixed_cost):
    print(get_dist_model(dists, transport_cost, fixed_cost, demand))
    
interact(get_dist_model_interact, fixed_cost=widgets.IntSlider(min=1,max=800000,step=20000,value=100000))

['Athens', 'Brussels', 'Hamburg', 'Madrid', 'Munich', 'Naples', 'Turin']


<function __main__.get_dist_model_interact>